In [1]:
import requests
import pandas as pd
import numpy as np
import json

In [2]:
url = "https://dummyjson.com/products/2"
response = requests.get(url)
if response.status_code == 200:
    data_products=response.json()
    print(data_products)
else:
    print(f"Failed to get data")
#Normalizing the JSOn to flat table and writing to a dataframe
df_product= pd.json_normalize(data_products)
df_product

{'id': 2, 'title': 'iPhone X', 'description': 'SIM-Free, Model A19211 6.5-inch Super Retina HD display with OLED technology A12 Bionic chip with ...', 'price': 899, 'discountPercentage': 17.94, 'rating': 4.44, 'stock': 34, 'brand': 'Apple', 'category': 'smartphones', 'thumbnail': 'https://i.dummyjson.com/data/products/2/thumbnail.jpg', 'images': ['https://i.dummyjson.com/data/products/2/1.jpg', 'https://i.dummyjson.com/data/products/2/2.jpg', 'https://i.dummyjson.com/data/products/2/3.jpg', 'https://i.dummyjson.com/data/products/2/thumbnail.jpg']}


,id,title,description,price,discountPercentage,rating,stock,brand,category,thumbnail,images
0,2,iPhone X,"SIM-Free, Model A19211 6.5-inch Super Retina H...",899,17.94,4.44,34,Apple,smartphones,https://i.dummyjson.com/data/products/2/thumbn...,[https://i.dummyjson.com/data/products/2/1.jpg...


In [5]:
data = [{'state': 'Florida', 
          'shortname': 'FL',
          'info': {'governor': 'Rick Scott'},
          'counties': [{'name': 'Dade', 'population': 12345},
                       {'name': 'Broward', 'population': 40000},
                       {'name': 'Palm Beach', 'population': 60000}]},
         {'state': 'Ohio',
          'shortname': 'OH',
          'info': {'governor': 'John Kasich'},
          'counties': [{'name': 'Summit', 'population': 1234},
                       {'name': 'Cuyahoga', 'population': 1337}]}]

df= pd.json_normalize(data, record_path=['counties'], meta=['state','shortname', ['info', 'governor']])
df

,name,population,state,shortname,info.governor
0,Dade,12345,Florida,FL,Rick Scott
1,Broward,40000,Florida,FL,Rick Scott
2,Palm Beach,60000,Florida,FL,Rick Scott
3,Summit,1234,Ohio,OH,John Kasich
4,Cuyahoga,1337,Ohio,OH,John Kasich


In [105]:
try:
    df= pd.json_normalize(data, record_path1=['counties'], meta=['state','shortname', ['info', 'governor']])
except json.JSONDecodeError:
        print("Error: The JSON document could not be decoded.")
except TypeError:
    print("Error: Invalid operation or function argument type.")
except KeyError:
    print("Error: One or more keys do not exist.")
except ValueError:
    print("Error: Invalid value detected.")
except Exception as e:
    # Catch any other exceptions
    print(f"An unexpected error occurred: {str(e)}")

Error: Invalid operation or function argument type.


In [36]:
def create_nested_dataframe_withKeys(data, key, nested_column):
    try:
        rows = []
        for entry in data:
            for nested_row in entry[nested_column]:
                print(type(nested_row))
                row = nested_row.copy()
                row[key] = entry[key]
                rows.append(row)

        return rows
    except json.JSONDecodeError:
        print("Error: The JSON document could not be decoded.")
    except TypeError:
        print("Error: Invalid operation or function argument type.")
    except KeyError:
        print("Error: One or more keys do not exist.")
    except ValueError:
        print("Error: Invalid value detected.")
    except Exception as e:
        # Catch any other exceptions
        print(f"An unexpected error occurred: {str(e)}")

In [37]:
df= pd.json_normalize(multi_nested_data, sep='_')
df

,class,student count,room,students,info_teachers_CS,info_teachers_AI,info_teachers_Neuro Science,info_teachers_physics
0,Year 1,20,Yellow,"[{'name': 'Tom', 'sex': 'M', 'grades': {'CS': ...",David Mulan,Andrew Ng,Huberman,NaN
1,Year 2,25,Blue,"[{'name': 'Tony', 'sex': 'M', 'grades': {'phys...",NaN,Alex Fredman,NaN,Richard Fennyman


In [38]:
df= df.drop('students', axis=1)
df

,class,student count,room,info_teachers_CS,info_teachers_AI,info_teachers_Neuro Science,info_teachers_physics
0,Year 1,20,Yellow,David Mulan,Andrew Ng,Huberman,NaN
1,Year 2,25,Blue,NaN,Alex Fredman,NaN,Richard Fennyman


In [39]:
#extractimg just the student dictionary from the multi nested json containing list and dictionaries
def extact_dictionary_from_nested_json(data,  nested_column):
    rows = []
    for entry in data:
        for student in entry[nested_column]:
            row = student.copy()            
            rows.append(row)    
    return  rows
students=extact_dictionary_from_nested_json(multi_nested_data,  'students')
students

[{'name': 'Tom',
  'sex': 'M',
  'grades': {'CS': 'A', 'AI': 'B', 'Neuro Science': 'B'}},
 {'name': 'James',
  'sex': 'M',
  'grades': {'CS': 'B', 'AI': 'B', 'Neuro Science': 'A'}},
 {'name': 'Tony', 'sex': 'M', 'grades': {'physics': 'B', 'AI': 'A'}},
 {'name': 'Jacqueline', 'sex': 'F', 'grades': {'physics': 'A', 'AI': 'A'}}]

In [40]:
#extrcat student along with class
students=create_nested_dataframe_withKeys(multi_nested_data, 'class', 'students')
df_grade = pd.json_normalize(students, sep="_")
df_grade

<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>


,name,sex,class,grades_CS,grades_AI,grades_Neuro Science,grades_physics
0,Tom,M,Year 1,A,B,B,NaN
1,James,M,Year 1,B,B,A,NaN
2,Tony,M,Year 2,NaN,A,NaN,B
3,Jacqueline,F,Year 2,NaN,A,NaN,A


In [41]:
df_full=pd.merge(df, df_grade, on='class')
df_full

,class,student count,room,info_teachers_CS,info_teachers_AI,info_teachers_Neuro Science,info_teachers_physics,name,sex,grades_CS,grades_AI,grades_Neuro Science,grades_physics
0,Year 1,20,Yellow,David Mulan,Andrew Ng,Huberman,NaN,Tom,M,A,B,B,NaN
1,Year 1,20,Yellow,David Mulan,Andrew Ng,Huberman,NaN,James,M,B,B,A,NaN
2,Year 2,25,Blue,NaN,Alex Fredman,NaN,Richard Fennyman,Tony,M,NaN,A,NaN,B
3,Year 2,25,Blue,NaN,Alex Fredman,NaN,Richard Fennyman,Jacqueline,F,NaN,A,NaN,A


In [56]:
nested_data = [
    { 
        'class': 'Year 1', 
        'student count': 20, 
        'room': 'Yellow',
        'info': {
            'teachers': { 
                'math': 'Rick Scott', 
                'physics': 'Madam Curie',
            }
        },
        'students': [
            { 'name': 'Tom', 'sex': 'M' },
            { 'name': 'James', 'sex': 'M' },
        ]
    },
    { 
        'class': 'Year 2', 
        'student count': 25, 
        'room': 'Blue',
        'info': {
            'teachers': { 
                 # no math teacher
                 'physics': 'Albert Einstein'
            }
        },
        'students': [
            { 'name': 'Tony', 'sex': 'M' },
            { 'name': 'Jacqueline', 'sex': 'F' },
        ]
    },
]

In [60]:
df=pd.json_normalize(nested_data, record_path=['students'], meta=['class', 'student count', 'room',['info', 'teachers']])
df

,name,sex,class,student count,room,info.teachers
0,Tom,M,Year 1,20,Yellow,"{'math': 'Rick Scott', 'physics': 'Madam Curie'}"
1,James,M,Year 1,20,Yellow,"{'math': 'Rick Scott', 'physics': 'Madam Curie'}"
2,Tony,M,Year 2,25,Blue,{'physics': 'Albert Einstein'}
3,Jacqueline,F,Year 2,25,Blue,{'physics': 'Albert Einstein'}


In [58]:
def flatten_json(y):
    try:
        out = {}

        def flatten(x, name=''):
            if type(x) is dict:
                for a in x:
                    flatten(x[a], name + a + '_')
            elif type(x) is list:
                i = 0
                for a in x:
                    flatten(a, name + str(i) + '_')
                    i += 1
            else:
                out[name[:-1]] = x

        flatten(y)
        return out
    except json.JSONDecodeError:
        print("Error: The JSON document could not be decoded.")
    except TypeError:
        print("Error: Invalid operation or function argument type.")
    except KeyError:
        print("Error: One or more keys do not exist.")
    except ValueError:
        print("Error: Invalid value detected.")
    except Exception as e:
        # Catch any other exceptions
        print(f"An unexpected error occurred: {str(e)}")

    

In [59]:
#Flatten nested dictionaries and lists
data_flattened = [flatten_json(class_info) for class_info in nested_data]

# Create DataFrame from flattened JSON
df = pd.DataFrame(data_flattened)

df

,class,student count,room,info_teachers_math,info_teachers_physics,students_0_name,students_0_sex,students_1_name,students_1_sex
0,Year 1,20,Yellow,Rick Scott,Madam Curie,Tom,M,James,M
1,Year 2,25,Blue,NaN,Albert Einstein,Tony,M,Jacqueline,F


In [61]:
multi_nested_data= [
    { 
        'class': 'Year 1', 
        'student count': 20, 
        'room': 'Yellow',
        'info': {
            'teachers': { 
                'CS': 'David Mulan', 
                'AI': 'Andrew Ng',
                'Neuro Science': 'Huberman'
            }
        },
        'students': [
            { 
                'name': 'Tom', 
                'sex': 'M', 
                'grades': {
                    'CS': 'A',
                    'AI': 'B',
                    'Neuro Science': 'B',
                }
            },
            { 
                'name': 'James', 
                'sex': 'M',
                'grades': {
                    'CS': 'B',
                    'AI': 'B',
                    'Neuro Science': 'A',
                }
            },
        ]
    },
    { 
        'class': 'Year 2', 
        'student count': 25, 
        'room': 'Blue',
        'info': {
            'teachers': { 
                'physics': 'Richard Fennyman',
                'AI':'Alex Fredman',
            }
        },
        'students': [
            { 
                'name': 'Tony', 
                'sex': 'M',
                'grades': {
                    'physics': 'B',
                    'AI': 'A'
                }
            },
            { 
                'name': 'Jacqueline', 
                'sex': 'F',
                'grades': {
                    'physics': 'A',
                    'AI': 'A'
                }
            },
        ]
    },
]

In [101]:
from IPython.display import display
# Flatten the data
flat_json_data = [flatten_json(record) for record in multi_nested_data]

# Convert flattened data to dataframe
df = pd.DataFrame(flat_json_data)
#pd.set_option('display.max_columns', None)
#print(df)
pd.options.display.max_columns = None
display(df)


,class,student count,room,info_teachers_CS,info_teachers_AI,info_teachers_Neuro Science,students_0_name,students_0_sex,students_0_grades_CS,students_0_grades_AI,students_0_grades_Neuro Science,students_1_name,students_1_sex,students_1_grades_CS,students_1_grades_AI,students_1_grades_Neuro Science,info_teachers_physics,students_0_grades_physics,students_1_grades_physics
0,Year 1,20,Yellow,David Mulan,Andrew Ng,Huberman,Tom,M,A,B,B,James,M,B,B,A,NaN,NaN,NaN
1,Year 2,25,Blue,NaN,Alex Fredman,NaN,Tony,M,NaN,A,NaN,Jacqueline,F,NaN,A,NaN,Richard Fennyman,B,A


In [74]:
df= pd.json_normalize(multi_nested_data, record_path=['students'], meta=['class','student count', 'room', ['info']])

df

,name,sex,grades.CS,grades.AI,grades.Neuro Science,grades.physics,class,student count,room,info
0,Tom,M,A,B,B,NaN,Year 1,20,Yellow,"{'teachers': {'CS': 'David Mulan', 'AI': 'Andr..."
1,James,M,B,B,A,NaN,Year 1,20,Yellow,"{'teachers': {'CS': 'David Mulan', 'AI': 'Andr..."
2,Tony,M,NaN,A,NaN,B,Year 2,25,Blue,"{'teachers': {'physics': 'Richard Fennyman', '..."
3,Jacqueline,F,NaN,A,NaN,A,Year 2,25,Blue,"{'teachers': {'physics': 'Richard Fennyman', '..."


In [97]:
df= df[['class', 'student count', 'room', 'name', 'sex', 
        'grades.CS', 'grades.AI','grades.physics','grades.Neuro Science']]
df

,class,student count,room,name,sex,grades.CS,grades.AI,grades.physics,grades.Neuro Science
0,Year 1,20,Yellow,Tom,M,A,B,NaN,B
1,Year 1,20,Yellow,James,M,B,B,NaN,A
2,Year 2,25,Blue,Tony,M,NaN,A,B,NaN
3,Year 2,25,Blue,Jacqueline,F,NaN,A,A,NaN


In [88]:
def create_nested_dataframe_withKeys(data, key, nested_column):
    try:
        rows = []
        for entry in data:
            print(entry)
            for nested_row in entry[nested_column]:
                print(nested_row)
                row = nested_row.copy()
                row[key] = entry[key]
                rows.append(row)

        return rows
    except json.JSONDecodeError:
        print("Error: The JSON document could not be decoded.")
    except TypeError:
        print("Error: Invalid operation or function argument type.")
    except KeyError:
        print("Error: One or more keys do not exist.")
    except ValueError:
        print("Error: Invalid value detected.")
    except Exception as e:
        # Catch any other exceptions
        print(f"An unexpected error occurred: {str(e)}")

In [92]:
def create_nested_dataframe_withKeys(data, key, nested_column):
    try:
        rows = []

        for entry in data:

            if isinstance(entry[nested_column], dict):  # If the nested_column is a dictionary
                row = entry[nested_column].copy()  # Copy the nested dictionary
                row[key] = entry[key]  # Add the key-value pair
                rows.append(row)  # Append it to the rows list
            else:
                for nested_row in entry[nested_column]:  # If the nested_column is a list

                    row = nested_row.copy()
                    row[key] = entry[key]
                    rows.append(row)

        return rows
    except json.JSONDecodeError:
        print("Error: The JSON document could not be decoded.")
    except TypeError:
        print("Error: Invalid operation or function argument type.")
    except KeyError:
        print("Error: One or more keys do not exist.")
    except ValueError:
        print("Error: Invalid value detected.")
    except Exception as e:
        # Catch any other exceptions
        print(f"An unexpected error occurred: {str(e)}")

In [94]:
teachers=create_nested_dataframe_withKeys(multi_nested_data,  'class', 'info')
teachers

[{'teachers': {'CS': 'David Mulan',
   'AI': 'Andrew Ng',
   'Neuro Science': 'Huberman'},
  'class': 'Year 1'},
 {'teachers': {'physics': 'Richard Fennyman', 'AI': 'Alex Fredman'},
  'class': 'Year 2'}]

In [98]:
df_teachers = pd.json_normalize(teachers, sep="_")
df_teachers

,class,teachers_CS,teachers_AI,teachers_Neuro Science,teachers_physics
0,Year 1,David Mulan,Andrew Ng,Huberman,NaN
1,Year 2,NaN,Alex Fredman,NaN,Richard Fennyman


In [99]:
df_full=pd.merge(df, df_teachers, on='class')
df_full

,class,student count,room,name,sex,grades.CS,grades.AI,grades.physics,grades.Neuro Science,teachers_CS,teachers_AI,teachers_Neuro Science,teachers_physics
0,Year 1,20,Yellow,Tom,M,A,B,NaN,B,David Mulan,Andrew Ng,Huberman,NaN
1,Year 1,20,Yellow,James,M,B,B,NaN,A,David Mulan,Andrew Ng,Huberman,NaN
2,Year 2,25,Blue,Tony,M,NaN,A,B,NaN,NaN,Alex Fredman,NaN,Richard Fennyman
3,Year 2,25,Blue,Jacqueline,F,NaN,A,A,NaN,NaN,Alex Fredman,NaN,Richard Fennyman
